In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import re

In [ ]:
df = pd.read_csv('../data/raw/phrasebank_raw.csv')
df

In [ ]:
df['sentence'] = df['sentence'].str.lower()
df

In [ ]:
df['sentence'] = df['sentence'].apply(lambda x: re.sub(r'\d+[\.,]?\d*', '<NUM>', x))
df.to_csv("../data/raw/phrasebank_raw.csv", index=False)

In [ ]:
df